<a href="https://colab.research.google.com/github/sanjana-singamsetty/Finance-expense-tracker/blob/main/capstone_banks_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Users upload a PDF via the React frontend.
- The PDF is sent to the Node.js server, which forwards it to the Flask service.
- The Flask service processes the PDF and returns the results.
- The React frontend displays the processed data in a table.








In [2]:
pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.3 MB/s eta 0:00:00


In [3]:
import tabula
import pandas as pd
from tabula import read_pdf
import os
import re
#import pyPdf
#import spacy

In [8]:
import tabula
import pandas as pd

# Dictionary for bank-specific column mapping
Bank_statement_features = {
    'HDFC': {
        'Date': 0,
        'Narration': 1,
        'Chq.': 2,
        'Value Dt': 3,
        'Withdrawal Amt.': 4,
        'Deposit Amt.': 5,
        'Closing Balance': 6,
    },
    'ICICI': {
        'Date': 0,
        'Particulars': 1,
        'Chq.No.': 2,
        'Withdrawals': 3,
        'Deposits': 4,
        'Auto Sweep': 5,
        'Reverse Sweep': 6,
        'Balance(INR)': 7,
    }
}

# Filepath to the PDF
filepath = r"/content/ICICI_Bank_Statement_New.pdf"

# Reading the PDF and extracting tables
rows_list = tabula.read_pdf(
    filepath,
    pages='all',
    silent=True,
    stream=True,
    lattice=True,
    guess=False,
    multiple_tables=True,
    area=(320.0, 9.0, 743.81, 601.11),
    encoding='utf-8'
)

# Combine tables into a single DataFrame
if isinstance(rows_list, list) and len(rows_list) > 0:
    rows_df = pd.concat(rows_list, ignore_index=True)
else:
    raise ValueError("No tables found in the PDF")

# Define column names
columns = ['Date', 'Particulars', 'Chq.No.', 'Withdrawals', 'Deposits', 'Auto Sweep', 'Reverse Sweep', 'Balance(INR)']
rows_df.columns = columns[:len(rows_df.columns)]  # Adjust column count dynamically

# Remove unnecessary columns
rows_df.drop(['Auto Sweep', 'Reverse Sweep'], axis=1, inplace=True, errors='ignore')

# Replace commas in numeric columns and convert to float
for col in ['Withdrawals', 'Deposits']:
    rows_df[col] = rows_df[col].str.replace(',', '', regex=True).astype(float, errors='ignore')

# Initialize 'Credit' and 'Debit' columns
rows_df['Credit'] = ''
rows_df['Debit'] = ''

# Add credit/debit logic
for i in range(1, len(rows_df)):
    if rows_df.loc[i, 'Withdrawals'] > rows_df.loc[i, 'Deposits']:
        rows_df.loc[i, 'Credit'] = "No"
        rows_df.loc[i, 'Debit'] = "Yes"
    elif rows_df.loc[i, 'Withdrawals'] < rows_df.loc[i, 'Deposits']:
        rows_df.loc[i, 'Credit'] = "Yes"
        rows_df.loc[i, 'Debit'] = "No"

# Display the formatted DataFrame
print(rows_df)


          Date                                        Particulars  Chq.No.  \
0   01-07-2019                                                B/F      NaN   
1   01-07-2019                                 BY CASH  - BIJOLIA      NaN   
2   02-07-2019                          BIL/INFT/001743804629/NA/      NaN   
3   02-07-2019  MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...      NaN   
4   03-07-2019                                   BY CASH-KACHHOLA      NaN   
5   03-07-2019  MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF\r...      NaN   
6   05-07-2019                     CLG/COSMOS   ENTERPRISES  /BRK   5181.0   
7   06-07-2019  MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF\r...      NaN   
8   08-07-2019  MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF\r...      NaN   
9   08-07-2019                          BIL/INFT/001748219433/NA/      NaN   
10  08-07-2019  UPI/918911468469/UPI/nandkishormali4/State Ban...      NaN   
11  09-07-2019  UPI/919006529705/Palak2vd2h/urmilarankaasin/Payt

In [ ]:
import pandas as pd
from tabula import read_pdf

filepath = r"C:\Users\nanda\Downloads\ICICI_Bank_Statement_New.pdf"

# Read all pages from the PDF
aux_list = tabula.read_pdf(filepath,
                           guess=False, pages='all', stream=False, silent=True, encoding="utf-8",
                           multiple_tables=True, area=(320.0, 9.0, 743.81, 601.11))

# Combine all tables into a single dataframe
aux_df = pd.concat(aux_list, ignore_index=True)



# Replace commas in numeric columns
aux_df.loc[:, "Withdrawals"] = aux_df["Withdrawals"].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)
aux_df.loc[:, "Deposits"] = aux_df["Deposits"].apply(lambda x: x.replace(',', '') if isinstance(x, str) else x)

# Convert columns to numeric
aux_df["Withdrawals"] = pd.to_numeric(aux_df["Withdrawals"], downcast="float", errors='coerce')
aux_df["Deposits"] = pd.to_numeric(aux_df["Deposits"], downcast="float", errors='coerce')

# Display the dataframe
print(aux_df)


          Date                                        Particulars  Chq.No.  \
0   01-07-2019                                                B/F      NaN   
1   01-07-2019                                 BY CASH  - BIJOLIA      NaN   
2   02-07-2019                          BIL/INFT/001743804629/NA/      NaN   
3   02-07-2019  MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...      NaN   
4   03-07-2019                                   BY CASH-KACHHOLA      NaN   
5   03-07-2019  MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF\r...      NaN   
6   05-07-2019                     CLG/COSMOS   ENTERPRISES  /BRK   5181.0   
7   06-07-2019  MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF\r...      NaN   
8   08-07-2019  MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF\r...      NaN   
9   08-07-2019                          BIL/INFT/001748219433/NA/      NaN   
10  08-07-2019  UPI/918911468469/UPI/nandkishormali4/State Ban...      NaN   
11  09-07-2019  UPI/919006529705/Palak2vd2h/urmilarankaasin/Payt

In [9]:
b_i = Bank_statement_features['ICICI']['Balance(INR)']
d_i = Bank_statement_features['ICICI']['Withdrawals']
w_i = Bank_statement_features['ICICI']['Deposits']

for i in range(1,len(rows_df)):
    if(rows_df['Withdrawals'][i]>rows_df['Deposits'][i]):
        rows_df['Credit'][i] = "No"
        rows_df['Debit'][i] = "Yes"

    if(rows_df['Withdrawals'][i]<rows_df['Deposits'][i]):
        rows_df['Credit'][i] = "Yes"
        rows_df['Debit'][i] = "No"

rows_df

<ipython-input-9-b8a01689359f>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rows_df['Credit'][i] = "Yes"
<ipython-input-9-b8a01689359f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

,Date,Particulars,Chq.No.,Withdrawals,Deposits,Balance(INR),Credit,Debit
0,01-07-2019,B/F,NaN,NaN,NaN,"16,674.45 Cr",,
1,01-07-2019,BY CASH - BIJOLIA,NaN,0.0,2000.0,"18,674.45 Cr",Yes,No
2,02-07-2019,BIL/INFT/001743804629/NA/,NaN,0.0,3740.0,"22,414.45 Cr",Yes,No
3,02-07-2019,MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...,NaN,0.0,3000.0,"25,414.45 Cr",Yes,No
4,03-07-2019,BY CASH-KACHHOLA,NaN,0.0,3000.0,"28,414.45 Cr",Yes,No
5,03-07-2019,MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,14000.0,"42,414.45 Cr",Yes,No
6,05-07-2019,CLG/COSMOS ENTERPRISES /BRK,5181.0,40000.0,0.0,"2,414.45 Cr",No,Yes
7,06-07-2019,MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.0,"7,414.45 Cr",Yes,No
8,08-07-2019,MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.0,"12,414.45 Cr",Yes,No
9,08-07-2019,BIL/INFT/001748219433/NA/,NaN,0.0,2150.0,"14,564.45 Cr",Yes,No


In [ ]:
# the following functions are basically to handel side cases and were not of use on the pdf which I used.

In [ ]:
## these two methods needed in case we have pdf containing multiple pages ##

#to count the no. of pages in a pdf
def count_pdf_pages(file_path):
    rxcountpages = re.compile(r"/Type\s*/Page([^s]|$)", re.MULTILINE|re.DOTALL)
    with open(file_path, "rb") as temp_file:
        return len(rxcountpages.findall(temp_file.read()))

n = count_pdf_pages("D:\Dataset\Loan2Grow\Pdf format\ICICI_Bank_Statement_New_1.pdf")
reader = pyPdf.PdfFileReader(open(filepath, mode='rb' ))
n = reader.getNumPages()

# if different pages have different areas from which data is to be extracted
df = []
for page in [str(i+1) for i in range(n)]:
    if page == "1":
            df.append(read_pdf(filepath, area=(320.0,9.0,743.81,601.11), pages=page))
    else:
            df.append(read_pdf(filepath, pages=page))



In [ ]:
## in case of ICICI bank not needed as they don't have unique headers on each page ##

# to remove the header from the list of transaction in case pdf have multiple pages
#(as bank statement have same header on each page)
# not of use in case of icici

Headers = []
# iterating over all rows extracted from the pdf
for i in rows_list:
    if if type(rows_list['Deposits'][i])==float:
        Headers.append(rows_list[i:i+1])
    else:
        break

# creating a list containing only valid transactions(removing the header rows from the extracted rows)
Valid_transactions = []
# iterating over all rows extracted from the pdf
for row in rows_lost:
    if row not in Headers:
        Valid_transactions.append(row)

In [ ]:
## here our code is already ignoring those rows which don't have dates therefore not needed ##

# To deal with those transactions which aquire multiple rows
#(when tabula is not able to differentiate whether multiple lines in a row belong to the same row
# and therfore would have created multiple rows for a single transaction)
# and these multiple lines is only because of narration/transaction_remarks column

single_row_transactions = []
#use the bank name according to pdf
index_date = Bank_statement_features['ICICI']['Date']
particulars_index = Bank_statement_features['ICICI']['Particulars']

for row in Valid_transactions:
    if row[index_date] is not None:                 #checking if the date exist in the row
        single_row_transactions.append(row)

    else:
        # adding the narration part of those rows as the last entry in single_row_transaction
        single_row_transactions[-1][narration_index] += row[narration_index]

In [ ]:
#can be used if needed
#categorizing transactions

def categorize(Particulars):
    word_list = tokenize(Particulars)

    if keyword in word_list:
        return category

    for rows in rows_list:
    rows_df['category'] = categorize(rows_df['Particulars'])